In [1]:
import pandas as pd
import numpy as np
import os


#получаем список файлов для чтения
all_files = os.listdir(path = 'C:\python39\PythonProjects\ParserMyScore')
needed_files = []
for file in all_files:
    if file in ('urls_all.csv', 'urls_new.csv', 'urls_refresh.csv', 'urls_new2.csv', 'urls_new3.csv', 
                'urls_new4.csv', 'urls_new5.csv', 'urls_new6.csv', 'urls_teams.csv', 'dict_league.csv', 
                'dict_team.csv', 'dict_league_main.csv'):
        continue
    elif file[-3:] == 'csv':
        needed_files.append(file)

df = pd.DataFrame()        
        
for file in needed_files:
    file_name = 'C:\python39\PythonProjects\ParserMyScore\\' + file
    df1 = pd.read_csv(file_name, sep = ';', encoding = 'ansi')
    df = pd.concat([df, df1])
    
df['country-league'] = df.country + ' ' + df.league
df.drop(['score_home', 'score_away', 'match_time'], axis=1, inplace=True)
df.rename(columns={'score_home_main_time': 'score_home', 'score_away_main_time': 'score_away'}, inplace=True)

df.match_date = df.match_date.apply(lambda x: x[0:16])
df.match_date = df.match_date.str.replace('29.02.', '28.02.', regex=False)
df.match_date = pd.to_datetime(df.match_date, format='%d.%m.%Y %H:%M')

#оставлю здесь код на память. Интересные возможности np.
'''conditions1 = [
    df['score_home'] == df['score_away'],
    df['score_home'] > df['score_away'],
    df['score_away'] > df['score_home']]
choices1 = ['Н', 'В', 'П']
conditions2 = [
    df['score_home'] == df['score_away'],
    df['score_home'] > df['score_away'],
    df['score_away'] > df['score_home']]
choices2 = ['Н', 'П', 'В']

df['home_result'] = np.select(conditions1, choices1)
df['away_result'] = np.select(conditions2, choices2)'''
    
df['home_result'] = ['Н' if x == y else 'В' if x > y else 'П' for  x, y in zip(df['score_home'], df['score_away'])]
df['away_result'] = ['Н' if x == y else 'П' if x > y else 'В' for  x, y in zip(df['score_home'], df['score_away'])]

df['home_result_score'] = [1 if x == y else 2 if x > y else 0 for  x, y in zip(df['score_home'], df['score_away'])]
df['away_result_score'] = [1 if x == y else 0 if x > y else 2 for  x, y in zip(df['score_home'], df['score_away'])]

df.sort_values('match_date', ascending=False, inplace=True, ignore_index=True)

In [2]:
df.shape

(137465, 15)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# df.score_home = df.score_home.str.replace('-', np.NaN, regex=False)
# df.score_away = df.score_home.str.replace('-', np.NaN, regex=False)
# df.dropna(subset=['score_home', 'score_away'], inplace=True)
df.drop(df[df.score_home == '-'].index, inplace=True)
df.score_home.astype(np.int8)
df.score_away.astype(np.int8)

0         0
1         1
2         2
3         0
4         1
         ..
137460    2
137461    3
137462    0
137463    0
137464    0
Name: score_away, Length: 137425, dtype: int8

In [5]:
dict_1 = pd.read_csv(r'C:\python39\PythonProjects\ParserMyScore\dict_league_main.csv', sep = ';', encoding = 'ansi')
dict_1.head()

,country-league,new_c_l
0,Австралия Лига А,Австралия Лига А
1,Азия Кубок Азии,Азия Кубок Азии
2,Азия Кубок Азии U23,Азия Кубок Азии U23
3,Азия Молодёжный чемпионат Азии U23,Азия Кубок Азии U23
4,Англия Blue Square Bet Premier,Англия Национальная лига


In [6]:
df = pd.merge(left=df, right=dict_1, how='left', left_on='country-league', right_on='country-league')
df.drop(['country-league', 'match_type', 'match_type_stage'], axis=1, inplace=True)
df.rename(columns={'new_c_l': 'country-league'}, inplace=True)

In [7]:
# df['matchdate'] = df.match_date.dt.date
# df['match_time'] = df.match_date.dt.time
df['month'] = pd.DatetimeIndex(df.match_date).month
# df['day'] = pd.DatetimeIndex(df.match_date).day
df['weekday'] = df.match_date.dt.dayofweek
df['hour'] = pd.DatetimeIndex(df.match_date).hour
# df['minute'] = pd.DatetimeIndex(df.match_date).minute

In [8]:
for i in range(1, 11):
    df[f'prev_{i}_home'] = df.groupby(['country-league', 'team_home'])['home_result_score'].shift(-i)
    df[f'prev_{i}_away'] = df.groupby(['country-league', 'team_away'])['away_result_score'].shift(-i)

In [9]:
for i in range(1, 6):
    df[f'prev_{i}_home_r'] = df.groupby(['country-league', 'team_home'])['home_result'].shift(-i)
    df[f'prev_{i}_away_r'] = df.groupby(['country-league', 'team_away'])['away_result'].shift(-i)

In [10]:
df['home_avg_scores'] = df[['prev_1_home', 'prev_2_home', 'prev_3_home', 'prev_4_home', 'prev_5_home',
                           'prev_6_home', 'prev_7_home', 'prev_8_home', 'prev_9_home', 'prev_10_home']].mean(axis=1)
df['away_avg_scores'] = df[['prev_1_away', 'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
                           'prev_6_away', 'prev_7_away', 'prev_8_away', 'prev_9_away', 'prev_10_away']].mean(axis=1)

In [11]:
df.dropna(how='any', inplace=True)

In [12]:
df.shape

(85029, 48)

In [ ]:
# df['home_avg_scores'] = df['home_avg_scores'].fillna(-1)
# df['away_avg_scores'] = df['away_avg_scores'].fillna(-1)

In [13]:
df['home_avg_scores'] = df['home_avg_scores'].apply(lambda x: round(x * 10, 0)).astype(np.int16)
df['away_avg_scores'] = df['away_avg_scores'].apply(lambda x: round(x * 10, 0)).astype(np.int16)

In [ ]:
# for i in range(1, 6):
#     df[f'prev_{i}_home'] = df[f'prev_{i}_home'].fillna(-1).astype(np.int16)
#     df[f'prev_{i}_away'] = df[f'prev_{i}_away'].fillna(-1).astype(np.int16)
#     df[f'prev_{i}_home_r'] = df[f'prev_{i}_home_r'].fillna('-')
#     df[f'prev_{i}_away_r'] = df[f'prev_{i}_away_r'].fillna('-')

In [14]:
for i in range(6,11):
    df.drop([f'prev_{i}_home'], axis=1, inplace=True)
    df.drop([f'prev_{i}_away'], axis=1, inplace=True)

In [15]:
df.head()

,match_date,team_home,team_away,score_home,score_away,season,country,league,home_result,away_result,home_result_score,away_result_score,country-league,month,weekday,hour,prev_1_home,prev_1_away,prev_2_home,prev_2_away,prev_3_home,prev_3_away,prev_4_home,prev_4_away,prev_5_home,prev_5_away,prev_1_home_r,prev_1_away_r,prev_2_home_r,prev_2_away_r,prev_3_home_r,prev_3_away_r,prev_4_home_r,prev_4_away_r,prev_5_home_r,prev_5_away_r,home_avg_scores,away_avg_scores
1,2022-06-30 14:30:00,Хэбэй Илиньшаньчжуан,Ухань,2,1,2022,Китай,Суперлига,В,П,2,0,Китай Суперлига,6,3,14,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,П,П,П,В,П,В,П,П,Н,П,8,8
7,2022-06-30 01:15:00,Серро Портеньо (Пар),Палмейрас (Бра),0,3,2022,Южная Америка,Кубок Либертадорес,П,В,0,2,Южная Америка Кубок Либертадорес,6,3,1,0.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,П,В,В,В,В,В,П,Н,В,Н,10,17
10,2022-06-29 22:30:00,Смуха,Аль-Ахли,3,2,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,22,1.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,2.0,2.0,Н,В,В,Н,П,Н,В,П,В,В,16,15
11,2022-06-29 20:00:00,Пирамидс,Энппи,1,0,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,20,2.0,1.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,1.0,В,Н,Н,Н,Н,В,П,В,В,Н,14,11
12,2022-06-29 15:30:00,Силвер Страйкерс,ТН Старс,4,1,2022,Малави,Суперлига,В,П,2,0,Малави Суперлига,6,2,15,1.0,0.0,2.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,Н,П,В,В,Н,В,В,П,Н,Н,16,7


In [16]:
df['prev_5_home_text'] = df.prev_1_home_r + df.prev_2_home_r + df.prev_3_home_r + df.prev_4_home_r + df.prev_5_home_r
df['prev_5_away_text'] = df.prev_1_away_r + df.prev_2_away_r + df.prev_3_away_r + df.prev_4_away_r + df.prev_5_away_r

In [17]:
for i in range(1,6):
    df.drop([f'prev_{i}_home_r'], axis=1, inplace=True)
    df.drop([f'prev_{i}_away_r'], axis=1, inplace=True)

In [18]:
df.head()

,match_date,team_home,team_away,score_home,score_away,season,country,league,home_result,away_result,home_result_score,away_result_score,country-league,month,weekday,hour,prev_1_home,prev_1_away,prev_2_home,prev_2_away,prev_3_home,prev_3_away,prev_4_home,prev_4_away,prev_5_home,prev_5_away,home_avg_scores,away_avg_scores,prev_5_home_text,prev_5_away_text
1,2022-06-30 14:30:00,Хэбэй Илиньшаньчжуан,Ухань,2,1,2022,Китай,Суперлига,В,П,2,0,Китай Суперлига,6,3,14,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,8,8,ППППН,ПВВПП
7,2022-06-30 01:15:00,Серро Портеньо (Пар),Палмейрас (Бра),0,3,2022,Южная Америка,Кубок Либертадорес,П,В,0,2,Южная Америка Кубок Либертадорес,6,3,1,0.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,10,17,ПВВПВ,ВВВНН
10,2022-06-29 22:30:00,Смуха,Аль-Ахли,3,2,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,22,1.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,2.0,2.0,16,15,НВПВВ,ВННПВ
11,2022-06-29 20:00:00,Пирамидс,Энппи,1,0,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,20,2.0,1.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,1.0,14,11,ВННПВ,ННВВН
12,2022-06-29 15:30:00,Силвер Страйкерс,ТН Старс,4,1,2022,Малави,Суперлига,В,П,2,0,Малави Суперлига,6,2,15,1.0,0.0,2.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,16,7,НВНВН,ПВВПН


In [ ]:
df.dtypes

In [ ]:
# df[(df['team_home'] == 'Адама Сити') & (df['league'] == 'Премьер-лига')][['team_home', 'team_away', 'prev_1_home', 'prev_2_home', 'prev_3_home', 'prev_4_home', 'prev_5_home', 
#    'prev_1_away', 'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
#    'home_avg_scores', 'away_avg_scores']].head(1)

In [ ]:
#кодировние данных
df['leave'] = df['leave'].map({'Very difficult': 0, 
                               'Somewhat difficult': 1, 
                               'Don\'t know': 2, 
                               'Somewhat easy': 3,
                               'Very easy': 4}) 

In [19]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['country-league']) 
df['country-league_digit'] = label_encoder.transform(df['country-league']) 
label_encoder.fit(df['country']) 
df['country_digit'] = label_encoder.transform(df['country']) 

In [28]:
df.head()

,match_date,team_home,team_away,score_home,score_away,season,country,league,home_result,away_result,home_result_score,away_result_score,country-league,month,weekday,hour,prev_1_home,prev_1_away,prev_2_home,prev_2_away,prev_3_home,prev_3_away,prev_4_home,prev_4_away,prev_5_home,prev_5_away,home_avg_scores,away_avg_scores,prev_5_home_text,prev_5_away_text,country-league_digit,country_digit
1,2022-06-30 14:30:00,Хэбэй Илиньшаньчжуан,Ухань,2,1,2022,Китай,Суперлига,В,П,2,0,Китай Суперлига,6,3,14,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,8,8,ППППН,ПВВПП,65,21
7,2022-06-30 01:15:00,Серро Портеньо (Пар),Палмейрас (Бра),0,3,2022,Южная Америка,Кубок Либертадорес,П,В,0,2,Южная Америка Кубок Либертадорес,6,3,1,0.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,10,17,ПВВПВ,ВВВНН,130,52
10,2022-06-29 22:30:00,Смуха,Аль-Ахли,3,2,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,22,1.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,2.0,2.0,16,15,НВПВВ,ВННПВ,49,13
11,2022-06-29 20:00:00,Пирамидс,Энппи,1,0,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,20,2.0,1.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,1.0,14,11,ВННПВ,ННВВН,49,13
12,2022-06-29 15:30:00,Силвер Страйкерс,ТН Старс,4,1,2022,Малави,Суперлига,В,П,2,0,Малави Суперлига,6,2,15,1.0,0.0,2.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,16,7,НВНВН,ПВВПН,72,25


In [25]:
df.away_avg_scores.unique()

array([ 8, 17, 15, 11,  7,  9, 12,  6, 10, 14,  5, 13,  1,  3,  2,  4, 16,
       18,  0, 19, 20], dtype=int16)

In [29]:
df = pd.get_dummies(df, columns=['prev_1_home', 'prev_1_away', 'prev_2_home', 'prev_2_away', 
                            'prev_3_home', 'prev_3_away', 'prev_4_home', 'prev_4_away', 'prev_5_home', 'prev_5_away'])

In [30]:
df.head()

,match_date,team_home,team_away,score_home,score_away,season,country,league,home_result,away_result,home_result_score,away_result_score,country-league,month,weekday,hour,home_avg_scores,away_avg_scores,prev_5_home_text,prev_5_away_text,country-league_digit,country_digit,prev_1_home_0.0,prev_1_home_1.0,prev_1_home_2.0,prev_1_away_0.0,prev_1_away_1.0,prev_1_away_2.0,prev_2_home_0.0,prev_2_home_1.0,prev_2_home_2.0,prev_2_away_0.0,prev_2_away_1.0,prev_2_away_2.0,prev_3_home_0.0,prev_3_home_1.0,prev_3_home_2.0,prev_3_away_0.0,prev_3_away_1.0,prev_3_away_2.0,prev_4_home_0.0,prev_4_home_1.0,prev_4_home_2.0,prev_4_away_0.0,prev_4_away_1.0,prev_4_away_2.0,prev_5_home_0.0,prev_5_home_1.0,prev_5_home_2.0,prev_5_away_0.0,prev_5_away_1.0,prev_5_away_2.0
1,2022-06-30 14:30:00,Хэбэй Илиньшаньчжуан,Ухань,2,1,2022,Китай,Суперлига,В,П,2,0,Китай Суперлига,6,3,14,8,8,ППППН,ПВВПП,65,21,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0
7,2022-06-30 01:15:00,Серро Портеньо (Пар),Палмейрас (Бра),0,3,2022,Южная Америка,Кубок Либертадорес,П,В,0,2,Южная Америка Кубок Либертадорес,6,3,1,10,17,ПВВПВ,ВВВНН,130,52,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0
10,2022-06-29 22:30:00,Смуха,Аль-Ахли,3,2,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,22,16,15,НВПВВ,ВННПВ,49,13,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1
11,2022-06-29 20:00:00,Пирамидс,Энппи,1,0,2021/2022,Египет,Премьер-лига,В,П,2,0,Египет Премьер-лига,6,2,20,14,11,ВННПВ,ННВВН,49,13,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0
12,2022-06-29 15:30:00,Силвер Страйкерс,ТН Старс,4,1,2022,Малави,Суперлига,В,П,2,0,Малави Суперлига,6,2,15,16,7,НВНВН,ПВВПН,72,25,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0


In [31]:
df.to_csv('calculated_games_clear.csv', sep=';', mode='w')

In [ ]:
pip install numpy --upgrade

In [ ]:
# import pandas as pd
# import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

# X = df[['prev_1_home', 'prev_2_home', 'prev_3_home', 'prev_4_home', 'prev_5_home',
#         'prev_1_away', 'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
#         'home_avg_scores', 'away_avg_scores']]
# Y = df['home_result_score']

X = df[['home_avg_scores', 'away_avg_scores']]
Y = df['home_result_score']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
model = RandomForestClassifier(n_estimators=101)  # , max_depth=8 , max_depth=9, max_features=10
#max_depth=9, max_features=10 5959
model.fit(X_train, y_train)

In [ ]:
r2_score(y_test, model.predict(X_test))

In [ ]:
accuracy_score(y_test, model.predict(X_test))

In [ ]:
import joblib
filename = 'RFC_model_457.sav'
joblib.dump(model, filename)

In [ ]:
# import pandas as pd
# import numpy as np
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score
# from sklearn.metrics import accuracy_score

# X = df[['prev_1_home', 'prev_2_home', 'prev_3_home', 'prev_4_home', 'prev_5_home',
#         'prev_1_away', 'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
#         'home_avg_scores', 'away_avg_scores']]
# Y = df['home_result_score']

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
model2 = KNeighborsClassifier(n_neighbors=20, weights='distance', leaf_size=12, algorithm='ball_tree') #distance  uniform 
model2.fit(X_train, y_train)

#n_neighbors=23, weights='distance', leaf_size=20, algorithm='ball_tree' 56,5

# algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
#leaf_sizeint, default=30
#Leaf size passed to BallTree or KDTree. 
#This can affect the speed of the construction and query, as well as the memory required to store the tree. 
#The optimal value depends on the nature of the problem.

In [ ]:
r2_score(y_test, model2.predict(X_test))

In [ ]:
accuracy_score(y_test, model2.predict(X_test))

In [ ]:
cols = ['prev_1_home', 'prev_2_home', 'prev_3_home', 'prev_4_home', 'prev_5_home', 
        'prev_1_away', 'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
        'home_avg_scores', 'away_avg_scores']


In [ ]:
m = [
[0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 130, 110],
[1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 50, 90],
[1, 2, 2, 2, 0, 0, 1, 0, 2, 1, 140, 110],
[1,2,2,2,-1,2,0,0,1,0,160,80],
[-1,-1,-1,-1,-1,2,1,2,0,0,200,125],
[0,0,2,1,2,2,0,0,0,1,130,100],
[1,1,1,0,1,2,1,1,1,1,90,110],
[0,2,1,2,1,0,0,2,2,1,110,70],
[2,0,-1,-1,-1,2,2,-1,-1,-1,100,167],
[1,1,-1,-1,-1,0,2,-1,-1,-1,133,67],
[2,0,2,-1,-1,2,0,1,-1,-1,100,125],
[0,1,1,1,2,1,2,2,0,2,80,130],
[0,2,2,1,-1,2,0,2,1,2,140,114],
[0,2,0,0,0,1,1,0,0,0,50,80],
[0,2,1,1,0,0,1,1,0,1,80,50],
]
df_2 = pd.DataFrame(m, columns=cols)
df_2

In [ ]:
model.predict(df_2)

In [ ]:
model2.predict(df_2)

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
importances

ar_f=[0]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), X.columns[idx]])
print('Значимость признака:')
# ar_f.sort(reverse=True)
ar_f

In [ ]:
import joblib
filename = 'KNC_model_56.sav'
joblib.dump(model, filename)

In [ ]:
import joblib
# loaded_model_2 = joblib.load('KNC_model_56.sav')
loaded_model_1 = joblib.load('RFC_model_457.sav')

In [ ]:
loaded_model_1.predict(df_2)

In [ ]:
      [2, 1, 2, 1, 2, 0, 1, 1, 1, 2, 0, 1, 2, 1, 1]

In [ ]:
loaded_model_2.predict(df_2)

In [ ]:
accuracy_score(y_test, loaded_model.predict(X_test))

In [ ]:
df_conc = pd.concat([df.team_home, df.team_away])
df_test_fact_v = pd.factorize(df_conc)[0]
df_test_fact = pd.DataFrame({'team': df_conc, 'value': df_test_fact_v})
df_test_fact = df_test_fact[~(df_test_fact.duplicated())]

In [ ]:
df_test_fact.to_csv(r'C:\python39\PythonProjects\ParserMyScore\dict_team.csv', sep=';', mode='w', 
                   index=False, encoding = 'ansi')

In [ ]:
dict_league = pd.Series(df['country-league'].unique().tolist())
dict_league

dict_league.to_csv(r'C:\python39\PythonProjects\ParserMyScore\dict_league.csv', sep=';', mode='w', 
                   index=False, encoding = 'ansi')

Создать функцию, которая будет принимать датафрейм (по умолчанию df), и серию. Серию передавать по каждой команде, проверять на условие и записывать в датафрейм данные.

In [ ]:
df['prev_1_home'] = df.groupby(['country-league', 'team_home'])['home_result_score'].shift(-1)
df['prev_2_home'] = df.groupby(['country-league', 'team_home'])['home_result_score'].shift(-2)
df['prev_3_home'] = df.groupby(['country-league', 'team_home'])['home_result_score'].shift(-3)
df['prev_4_home'] = df.groupby(['country-league', 'team_home'])['home_result_score'].shift(-4)
df['prev_5_home'] = df.groupby(['country-league', 'team_home'])['home_result_score'].shift(-5)

df['prev_1_away'] = df.groupby(['country-league', 'team_away'])['away_result_score'].shift(-1)
df['prev_2_away'] = df.groupby(['country-league', 'team_away'])['away_result_score'].shift(-2)
df['prev_3_away'] = df.groupby(['country-league', 'team_away'])['away_result_score'].shift(-3)
df['prev_4_away'] = df.groupby(['country-league', 'team_away'])['away_result_score'].shift(-4)
df['prev_5_away'] = df.groupby(['country-league', 'team_away'])['away_result_score'].shift(-5)

In [ ]:
df['prev_1_home_team_score_home'] = df.groupby(['country-league', 'team_home'])['score_home'].shift(-1)
df['prev_1_home_team_score_away'] = df.groupby(['country-league', 'team_home'])['score_away'].shift(-1)
df['prev_2_home_team_score_home'] = df.groupby(['country-league', 'team_home'])['score_home'].shift(-2)
df['prev_2_home_team_score_away'] = df.groupby(['country-league', 'team_home'])['score_away'].shift(-2)
df['prev_3_home_team_score_home'] = df.groupby(['country-league', 'team_home'])['score_home'].shift(-3)
df['prev_3_home_team_score_away'] = df.groupby(['country-league', 'team_home'])['score_away'].shift(-3)
df['prev_4_home_team_score_home'] = df.groupby(['country-league', 'team_home'])['score_home'].shift(-4)
df['prev_4_home_team_score_away'] = df.groupby(['country-league', 'team_home'])['score_away'].shift(-4)
df['prev_5_home_team_score_home'] = df.groupby(['country-league', 'team_home'])['score_home'].shift(-5)
df['prev_5_home_team_score_away'] = df.groupby(['country-league', 'team_home'])['score_away'].shift(-5)

df['prev_1_away_team_score_home'] = df.groupby(['country-league', 'team_away'])['score_home'].shift(-1)
df['prev_1_away_team_score_away'] = df.groupby(['country-league', 'team_away'])['score_away'].shift(-1)
df['prev_2_away_team_score_home'] = df.groupby(['country-league', 'team_away'])['score_home'].shift(-2)
df['prev_2_away_team_score_away'] = df.groupby(['country-league', 'team_away'])['score_away'].shift(-2)
df['prev_3_away_team_score_home'] = df.groupby(['country-league', 'team_away'])['score_home'].shift(-3)
df['prev_3_away_team_score_away'] = df.groupby(['country-league', 'team_away'])['score_away'].shift(-3)
df['prev_4_away_team_score_home'] = df.groupby(['country-league', 'team_away'])['score_home'].shift(-4)
df['prev_4_away_team_score_away'] = df.groupby(['country-league', 'team_away'])['score_away'].shift(-4)
df['prev_5_away_team_score_home'] = df.groupby(['country-league', 'team_away'])['score_home'].shift(-5)
df['prev_5_away_team_score_away'] = df.groupby(['country-league', 'team_away'])['score_away'].shift(-5)

In [ ]:
df['home_avg_scores'] = df.sort_values('match_date', ascending=True)\
    .groupby(['country-league', 'team_home'])['home_result_score'].rolling(10, min_periods=1).mean()\
    .reset_index(level=[0,1], drop=True)
df['away_avg_scores'] = df.sort_values('match_date', ascending=True)\
    .groupby(['country-league', 'team_away'])['away_result_score'].rolling(10, min_periods=1).mean()\
    .reset_index(level=[0,1], drop=True)

In [ ]:
df_test_fact = pd.read_csv(r'C:\python39\PythonProjects\ParserMyScore\dict_team.csv', sep = ';', encoding = 'ansi')
df_test_fact.head()

In [ ]:
df = pd.merge(left=df, right=df_test_fact, how='left', left_on='team_home', right_on='team')
df.drop(['team'], axis=1, inplace=True)

In [ ]:
df.columns = ['match_date', 'team_home', 'team_away', 'score_home', 'score_away',
       'season', 'country', 'league', 'home_result', 'away_result',
       'home_result_score', 'away_result_score', 'comp_type', 'country-league',
       'country_league_rank', 'comp_type_rank', 'country_rank', 'month', 'day',
       'weekday', 'hour', 'minute', 'prev_1_home', 'prev_2_home',
       'prev_3_home', 'prev_4_home', 'prev_5_home', 'prev_1_away',
       'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
       'prev_1_home_team_score_home', 'prev_1_home_team_score_away',
       'prev_2_home_team_score_home', 'prev_2_home_team_score_away',
       'prev_3_home_team_score_home', 'prev_3_home_team_score_away',
       'prev_4_home_team_score_home', 'prev_4_home_team_score_away',
       'prev_5_home_team_score_home', 'prev_5_home_team_score_away',
       'prev_1_away_team_score_home', 'prev_1_away_team_score_away',
       'prev_2_away_team_score_home', 'prev_2_away_team_score_away',
       'prev_3_away_team_score_home', 'prev_3_away_team_score_away',
       'prev_4_away_team_score_home', 'prev_4_away_team_score_away',
       'prev_5_away_team_score_home', 'prev_5_away_team_score_away',
       'home_avg_scores', 'away_avg_scores', 'team_home_d']

In [ ]:
df = pd.merge(left=df, right=df_test_fact, how='left', left_on='team_away', right_on='team')
df.drop(['team'], axis=1, inplace=True)

In [ ]:
df.columns = ['match_date', 'team_home', 'team_away', 'score_home', 'score_away',
       'season', 'country', 'league', 'home_result', 'away_result',
       'home_result_score', 'away_result_score', 'comp_type', 'country-league',
       'country_league_rank', 'comp_type_rank', 'country_rank', 'month', 'day',
       'weekday', 'hour', 'minute', 'prev_1_home', 'prev_2_home',
       'prev_3_home', 'prev_4_home', 'prev_5_home', 'prev_1_away',
       'prev_2_away', 'prev_3_away', 'prev_4_away', 'prev_5_away',
       'prev_1_home_team_score_home', 'prev_1_home_team_score_away',
       'prev_2_home_team_score_home', 'prev_2_home_team_score_away',
       'prev_3_home_team_score_home', 'prev_3_home_team_score_away',
       'prev_4_home_team_score_home', 'prev_4_home_team_score_away',
       'prev_5_home_team_score_home', 'prev_5_home_team_score_away',
       'prev_1_away_team_score_home', 'prev_1_away_team_score_away',
       'prev_2_away_team_score_home', 'prev_2_away_team_score_away',
       'prev_3_away_team_score_home', 'prev_3_away_team_score_away',
       'prev_4_away_team_score_home', 'prev_4_away_team_score_away',
       'prev_5_away_team_score_home', 'prev_5_away_team_score_away',
       'home_avg_scores', 'away_avg_scores', 'team_home_d', 'team_away_d']
df.team_away_d.astype(np.int16)

In [ ]:
df.season = df.season.apply(lambda x: int(str(x).split('/')[0]))

In [ ]:
df['prev_1_home'] = df['prev_1_home'].fillna(-1).astype(np.int16)
df['prev_2_home'] = df['prev_2_home'].fillna(-1).astype(np.int16)
df['prev_3_home'] = df['prev_3_home'].fillna(-1).astype(np.int16)
df['prev_4_home'] = df['prev_4_home'].fillna(-1).astype(np.int16)
df['prev_5_home'] = df['prev_5_home'].fillna(-1).astype(np.int16)

df['prev_1_away'] = df['prev_1_away'].fillna(-1).astype(np.int16)
df['prev_2_away'] = df['prev_2_away'].fillna(-1).astype(np.int16)
df['prev_3_away'] = df['prev_3_away'].fillna(-1).astype(np.int16)
df['prev_4_away'] = df['prev_4_away'].fillna(-1).astype(np.int16)
df['prev_5_away'] = df['prev_5_away'].fillna(-1).astype(np.int16)

In [ ]:
pd.set_option('display.max_rows', 300) #100